# Nemotron SDG – Brev deployment test

This notebook verifies your Brev Launchable by running **Data Designer** with **Nemotron** models.

- **Kernel:** Use **"Data Designer (Nemotron)"** (set in the top-right).
- **API key:** Set `NVIDIA_API_KEY` in the instance environment (from [build.nvidia.com](https://build.nvidia.com)).

In [ ]:
import os

if not os.environ.get("NVIDIA_API_KEY"):
    raise RuntimeError(
        "NVIDIA_API_KEY is not set. Set it in the Brev instance environment (or in this notebook) "
        "to call Nemotron on build.nvidia.com."
    )
print("NVIDIA_API_KEY is set.")

## Imports and Data Designer setup

In [ ]:
import data_designer.config as dd
from data_designer.interface import DataDesigner

data_designer = DataDesigner()

## Nemotron model config

We use **nvidia/nemotron-3-nano-30b-a3b** (default NVIDIA provider).

In [ ]:
MODEL_PROVIDER = "nvidia"
MODEL_ID = "nvidia/nemotron-3-nano-30b-a3b"
MODEL_ALIAS = "nemotron-nano-v3"

model_configs = [
    dd.ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=dd.ChatCompletionInferenceParams(
            temperature=1.0,
            top_p=1.0,
            max_tokens=512,
            extra_body={"chat_template_kwargs": {"enable_thinking": False}},
        ),
    )
]
config_builder = dd.DataDesignerConfigBuilder(model_configs=model_configs)

## Minimal schema: category + LLM-generated text

In [ ]:
config_builder.add_column(
    dd.SamplerColumnConfig(
        name="category",
        sampler_type=dd.SamplerType.CATEGORY,
        params=dd.CategorySamplerParams(values=["Electronics", "Books", "Home"]),
    )
)
config_builder.add_column(
    dd.LLMTextColumnConfig(
        name="product_name",
        prompt=(
            "Generate a short product name for the category: {{ category }}. "
            "Reply with only the product name, no quotes."
        ),
        model_alias=MODEL_ALIAS,
    )
)
data_designer.validate(config_builder)

## Run preview (2 records) with Nemotron

In [ ]:
preview = data_designer.preview(config_builder, num_records=2)
preview.display_sample_record()

In [ ]:
preview.dataset

## Done

If you see a small table with `category` and `product_name`, the Brev deployment and Nemotron integration are working. You can run more tutorials from `sdg/SDG_network/docs/notebook_source/` or use the CLI: `data-designer preview/create` from the `sdg/SDG_network` directory.